In [12]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, optimizers, Sequential, metrics

os.environ["TF_CPP_MIN_LOG_LEVEL"] = '2'

tf.random.set_seed(0)

net = [
    layers.Conv2D(6, kernel_size = [5, 5], padding = "same", activation = tf.nn.relu),
    layers.MaxPool2D(pool_size = [2, 2], strides = 2, padding = "same"),
    layers.Conv2D(16, kernel_size = [5, 5], padding = "same", activation = tf.nn.relu),
    layers.MaxPool2D(pool_size = [2, 2], strides = 2, padding = "same"),
]

fc = [
    layers.Dense(120, activation = tf.nn.relu),
    layers.Dense(84, activation = tf.nn.relu),
    layers.Dropout(rate = 0.5),
    layers.Dense(10, activation = None),
]

def process(x, y):
    x = tf.cast(x, dtype = tf.float32) / 255.
    y = tf.cast(y, dtype = tf.int32)
    return x, y

(x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()
y_train = tf.squeeze(y_train, axis = 1)
y_test = tf.squeeze(y_test, axis = 1)

train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.shuffle(1000).map(process).batch(64)

test_data = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_data = test_data.map(process).batch(64)

def main():
    lenet5 = Sequential(net)
    fc_net = Sequential(fc)
    
    lenet5.build(input_shape = [None, 32, 32, 3])
    fc_net.build(input_shape = [None, 1024])
    optimizer = optimizers.Adam(lr = 1e-4)
    
    variables = lenet5.trainable_variables + fc_net.trainable_variables
    for epoch in range(50):
        for step, (x, y) in enumerate(train_data):
            with tf.GradientTape() as tape:
                out = lenet5(x)
                out = tf.reshape(out, [-1, 1024])
                logits = fc_net(out)
                y_onehot = tf.one_hot(y, depth = 10)
                loss = tf.losses.categorical_crossentropy(y_onehot, logits, from_logits = True)
                loss = tf.reduce_mean(loss)
            grads = tape.gradient(loss, variables)
            optimizer.apply_gradients(zip(grads, variables))
            
            if step % 100 == 0:
                print(epoch, step, 'loss: ', float(loss))
        
        total = 0
        ac = 0
        for (x, y) in test_data:
            out = lenet5(x)
            out = tf.reshape(out, [-1, 1024])
            logits = fc_net(out)
            prob = tf.nn.softmax(logits, axis = 1)
            pred = tf.argmax(prob, axis = 1)
            pred = tf.cast(pred, dtype = tf.int32)
            correct = tf.cast(tf.equal(pred, y), dtype = tf.int32)
            correct = tf.reduce_sum(correct)
            total += x.shape[0]
            ac += int(correct)
        acc = ac / total
        print(epoch, 'accruacy: ', acc)
        
if __name__ == '__main__':
    main()

0 0 loss:  2.3012266159057617
0 100 loss:  2.257014751434326
0 200 loss:  2.0397090911865234
0 300 loss:  1.806654691696167
0 400 loss:  1.7996854782104492
0 500 loss:  1.8516989946365356
0 600 loss:  1.764099359512329
0 700 loss:  1.7259016036987305
0 accruacy:  0.4058
1 0 loss:  1.752593994140625
1 100 loss:  1.6205387115478516
1 200 loss:  1.6674103736877441
1 300 loss:  1.4940321445465088
1 400 loss:  1.5181968212127686
1 500 loss:  1.69584059715271
1 600 loss:  1.5777965784072876
1 700 loss:  1.3257876634597778
1 accruacy:  0.455
2 0 loss:  1.4320837259292603
2 100 loss:  1.4230303764343262
2 200 loss:  1.346543788909912
2 300 loss:  1.5770606994628906
2 400 loss:  1.5052869319915771
2 500 loss:  1.6831152439117432
2 600 loss:  1.5782365798950195
2 700 loss:  1.4543124437332153
2 accruacy:  0.4722
3 0 loss:  1.7353651523590088
3 100 loss:  1.6109898090362549
3 200 loss:  1.2862412929534912
3 300 loss:  1.4332849979400635
3 400 loss:  1.5409505367279053
3 500 loss:  1.4503920078277

29 500 loss:  0.7916538715362549
29 600 loss:  0.8887718319892883
29 700 loss:  0.991263747215271
29 accruacy:  0.6012
30 0 loss:  1.1337380409240723
30 100 loss:  0.7779334783554077
30 200 loss:  1.1077827215194702
30 300 loss:  0.8515812754631042
30 400 loss:  0.7062546014785767
30 500 loss:  0.9437724947929382
30 600 loss:  1.0530909299850464
30 700 loss:  1.001492977142334
30 accruacy:  0.6198
31 0 loss:  1.0651981830596924
31 100 loss:  0.9732367396354675
31 200 loss:  1.0899953842163086
31 300 loss:  0.7914661765098572
31 400 loss:  0.6508108377456665
31 500 loss:  1.1208500862121582
31 600 loss:  0.9009143114089966
31 700 loss:  0.8736617565155029
31 accruacy:  0.6182
32 0 loss:  0.9535546898841858
32 100 loss:  0.9147692918777466
32 200 loss:  0.9428251385688782
32 300 loss:  1.1126493215560913
32 400 loss:  0.9279395937919617
32 500 loss:  0.9911870956420898
32 600 loss:  0.7180837392807007
32 700 loss:  0.9657282829284668
32 accruacy:  0.6115
33 0 loss:  0.9347472190856934
33

Using **keras** to build a LeNet-5 model for cifar-10.  
$input = [32, 32, 3]$  
The first convolutional layer has $[5, 5, 6]$ kernels, using `SAME` convolution operation to get $[32, 32, 6]$ output  
Then comes the first sampling layer with max pooling, which makes the output $[16, 16, 6]$  
The second convolutional layer has $[5, 5, 16]$ kernels, using `SAME` convolution operation, and the output shape equals to $[16, 16, 16]$  
And the second sampling layer  also use max pooling to reshape the output to $[8, 8, 16]$  
Before the shape goes to the full connection layer we flatten it to a $[1, 8\times 8\times 16=1024]$ vector, and the first full connection layer has $120$ kernels while the second one has $84$ kernels.
Using dropout operation can avoid overfitting problem. 
In the output layer, we use *softmax* regression:
$$
softmax(z_i)=\frac{e^{z_i}}{\sum_j e^{z_j}}
$$
to get the output for the $10$ output channels, and reduce the mean of cross entropy loss function:
$$
J(W, b) = -\frac{1}{m}\sum_{i=1}^{m} y^{(i)}\ln\hat{y}^{(i)}
$$
where $y^{(i)}$ is the standerd label and $\hat{y}^{(i)}$ is the network's output.
We can get an accuary around $63\%$ after $50$ epochs with a mini-batch size of $64$.